<a href="https://colab.research.google.com/github/sanchitgulati/spotipy_journey_of_the_music/blob/main/spotipy_journey_of_the_music.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [104]:
import pip

def install(package):
    if hasattr(pip, 'main'):
        pip.main(['install', package])
    else:
        pip._internal.main(['install', package])



In [105]:
# %%capture captures the cell output, so need less things don't populate the cell 
%%capture
install('spotipy')

In [106]:
%%capture
install('panda')

In [107]:
%%capture
install('sklearn')

In [108]:
%%capture
install('matplotlib')

In [109]:
%%capture
install('seaborn')

In [110]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Imports

In [111]:
import pandas as pd
import datetime
import time

# Secret Key

In [112]:
secrets = {'CLIENT_ID': "<CLIENT-ID-HERE>",'CLIENT_SECRET':"<CLIENT-SECRET-HERE>"}

In [113]:
# Used once to store your APP secrets

# import pickle
# with open('secrets.dictionary', 'wb') as secret_dictionary_file:
#   pickle.dump(secrets, secret_dictionary_file)
# !cp 'secrets.dictionary' "drive/My Drive/spotipy/secrets.dictionary" 

In [114]:
import pickle
!cp "drive/My Drive/spotipy/secrets.dictionary" 'secrets.dictionary' 
with open('secrets.dictionary', 'rb') as secret_dictionary_file:
    secrets = pickle.load(secret_dictionary_file)

# Imports

In [115]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=secrets['CLIENT_ID'], client_secret=secrets['CLIENT_SECRET'])
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Functions

In [116]:
def GetNewResults(market,popularity = 70):
  data = []
  # Fetches only the top? 50 albums, as limited by the API for single call
  album_results = sp.search(q='tag:new', market=market,type='album', limit=50)
  for i, a_overview in enumerate(album_results['albums']['items']):
    a = GetAlbumResults(a_overview['id'])
    if a[-1] > popularity: # only extremely popular albums, defaults to 70
      data.append(a)
  return data      

In [117]:
def GetAlbumResults(album_id):
  a = sp.album(album_id)
  date_object = datetime.datetime.strptime(a['release_date'], '%Y-%m-%d').date()
  td = datetime.date.today() - date_object
  return [a['id'],a['name'],a['images'][1]['url'],a['release_date'],td.days,a['popularity']] 

# Main

In [118]:
# all_market_codes = sp.available_markets()
all_market_codes = {"markets":['IN','US','PK','BR','MX','JP','NL','FR','DE','FR']}
# all_market_codes = {"markets":['US']}

In [119]:
!cp "drive/My Drive/spotipy/data.csv" data.csv
df = pd.read_csv('data.csv')

In [120]:
data = []

unique_albums = df.id.unique()
for album_id in unique_albums:
  data.append(GetAlbumResults(album_id))


for market in all_market_codes['markets']:    
  d = GetNewResults(market)
  data.extend(d)


df = pd.DataFrame(data,columns=['id','name','image','release_date','days_since_release','popularity'])
df = df.drop_duplicates(subset=['id','days_since_release'])

In [121]:
df.to_csv('data.csv')
!cp data.csv "drive/My Drive/spotipy/"

# Future
## Analysis of collected data, need atleast 3 months of data
### Will CRON job the code above privately

In [122]:
## Retreive File(s) from Google drive

In [123]:
# !cp "drive/My Drive/spotipy/<file-name>.csv" data.csv 

In [124]:
# df = pd.read_csv('<file-name>.csv')

# ScratchPad

In [125]:
GetNewResults('IN')

[['0gr5OmB74UhoANEXwYT3gE',
  'Dreamers [Music from the FIFA World Cup Qatar 2022 Official Soundtrack]',
  'https://i.scdn.co/image/ab67616d00001e02ef57183066d6cac0cabb85c6',
  '2022-11-20',
  10,
  79]]

In [126]:
GetAlbumResults('1y0Yts8U11AdgMgl4tPDII')

['1y0Yts8U11AdgMgl4tPDII',
 'Autobahn',
 'https://i.scdn.co/image/ab67616d00001e02f123e039f29e747ae01e99a1',
 '2022-11-18',
 12,
 74]